In [7]:
import pandas as pd

# Load the CSV
df = pd.read_csv('donnees_Vols_BON_F.csv')  # use your actual filename if different
df['is_attack'] = 1
# List of columns to keep
columns_to_keep = [
    'mission_id', 'time', 'real_x', 'real_y', 'real_z',
    'vx', 'vy', 'vz', 'ax', 'ay', 'az',
    'roll', 'pitch', 'yaw',
    'wind_x', 'wind_y', 'wind_z',
    'erreur_m', 'velocity',
    'rain', 'fog','snow' ,'is_attack'      

]

# Filter the DataFrame
filtered_df = df[columns_to_keep]

# Save the result
filtered_df.to_csv('donnees_nettoyee_bon.csv', index=False)

print(" Saved cleaned CSV with selected columns to 'donnees_nettoyee.csv'")

 Saved cleaned CSV with selected columns to 'donnees_nettoyee.csv'


In [8]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('donnees_nettoyee_bon.csv')

# Final list of valid flight rows
output_rows = []

# Track current block of rows
current_block = []
current_id = None

# Counter for assigning new clean mission_ids
new_flight_id = 1

# Go through rows one by one
for _, row in df.iterrows():
    row_id = row['mission_id']

    if current_id is None:
        current_id = row_id
        current_block.append(row)
    elif row_id == current_id:
        current_block.append(row)
    else:
        # ID changed, time to process the current block
        if len(current_block) == 21:
            for r in current_block:
                r['mission_id'] = new_flight_id
                output_rows.append(r)
            new_flight_id += 1

        # Start a new block
        current_block = [row]
        current_id = row_id

# Handle the last block
if len(current_block) == 21:
    for r in current_block:
        r['mission_id'] = new_flight_id
        output_rows.append(r)

# Convert to DataFrame
output_df = pd.DataFrame(output_rows)

# Save to CSV
output_df.to_csv('donnees_vol_multiple_filtered_bon.csv', index=False)

print("✅ Saved cleaned flight data with new mission IDs to 'cleaned_flight_data.csv'")

✅ Saved cleaned flight data with new mission IDs to 'cleaned_flight_data.csv'


In [9]:
import pandas as pd

# Charger les données
df = pd.read_csv("donnees_vol_multiple_filtered_bon.csv")

# Liste des vols valides
vols_valides = []

# Vérification des conditions sur chaque vol
for mission_id, groupe in df.groupby("mission_id"):
    groupe_sorted = groupe.sort_values("time")
    premiers_wp = groupe_sorted.head(7)

    # Vérifier les deux conditions
    condition_erreur_7points = (premiers_wp["erreur_m"] > 8).any()
    moyenne_vol = groupe_sorted["erreur_m"].mean()
    condition_moyenne = moyenne_vol >= 18

    # Appliquer le OU : on supprime le vol si l'une des deux conditions est vraie
    if condition_erreur_7points or condition_moyenne:
        continue

    # Sinon, garder ce vol
    vols_valides.append(mission_id)

# Filtrer le DataFrame avec les vols valides
df_final = df[df["mission_id"].isin(vols_valides)]

# Sauvegarder dans un nouveau fichier CSV
df_final.to_csv("donnees_vols_valides.csv", index=False)

# Affichage de la moyenne d'erreur et du nombre de vols gardés
moyenne_finale = df_final["erreur_m"].mean()
print(f"Moyenne finale des erreurs : {moyenne_finale:.2f} mètres")
print(f"Nombre de vols restants : {len(vols_valides)}")
print("Fichier 'donnees_vols_valides.csv' enregistré avec succès.")


Moyenne finale des erreurs : 3.93 mètres
Nombre de vols restants : 168
Fichier 'donnees_vols_valides.csv' enregistré avec succès.


In [10]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import pandas as pd
import numpy as np

def main():
    # 1. Charger les données
    df = pd.read_csv('donnees_vols_valides.csv')

    # 2. Calcul de la matrice de corrélation
    corr = df.corr()
    print("Matrice de corrélation :")
    print(corr)
    print("\n" + "="*80 + "\n")

    # 3. Extraction des paires de variables (sans doublons ni diagonale)
    mask = np.triu(np.ones(corr.shape), k=1).astype(bool)
    corr_pairs = (
        corr.where(mask)
            .stack()
            .reset_index()
            .rename(columns={'level_0': 'Variable1', 'level_1': 'Variable2', 0: 'Corrélation'})
    )

    # 4. Filtrage des paires avec corrélation >= 0.7 en valeur absolue
    high_corr = corr_pairs[corr_pairs['Corrélation'].abs() >= 0.7] \
        .sort_values(by='Corrélation', ascending=False)

    # 5. Affichage des paires à forte corrélation
    print("Paires de variables avec |corrélation| ≥ 0.7 :")
    for _, row in high_corr.iterrows():
        v1, v2, r = row['Variable1'], row['Variable2'], row['Corrélation']
        print(f"  - {v1} ↔ {v2} : corrélation = {r:.3f}")

if __name__ == "__main__":
    main()


Matrice de corrélation :
            mission_id      time    real_x    real_y    real_z        vx  \
mission_id    1.000000 -0.596357 -0.030977 -0.028418 -0.181844 -0.013271   
time         -0.596357  1.000000  0.085639  0.043542  0.326117  0.094506   
real_x       -0.030977  0.085639  1.000000 -0.188681 -0.229465  0.221977   
real_y       -0.028418  0.043542 -0.188681  1.000000  0.123976 -0.007255   
real_z       -0.181844  0.326117 -0.229465  0.123976  1.000000 -0.095032   
vx           -0.013271  0.094506  0.221977 -0.007255 -0.095032  1.000000   
vy            0.006108  0.030758  0.000441  0.555169  0.017234  0.053949   
vz           -0.086812  0.081052  0.024514 -0.036809  0.638818 -0.130578   
ax            0.019252 -0.002652  0.022647  0.009463 -0.027100  0.056764   
ay            0.006180  0.020683  0.004977 -0.023283  0.061979 -0.006773   
az            0.051174 -0.023245 -0.059117 -0.023641  0.133729 -0.047733   
roll          0.114427 -0.114554 -0.162777 -0.030458  0.065601 

In [11]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import pandas as pd

def main():
    # 1. Charger les données
    df = pd.read_csv('donnees_vols_valides.csv')

    # 2. Calcul de la variance
    variances = df.var()

    # 3. Filtrer les variables avec une variance > 0.01
    filtered_vars = variances[variances > 0.01].sort_values(ascending=False)

    # 4. Affichage
    print("Variables avec une variance > 0.01 :\n")
    for var, var_value in filtered_vars.items():
        print(f"  - {var} : variance = {var_value:.4f}")

if __name__ == "__main__":
    main()


Variables avec une variance > 0.01 :

  - time : variance = 756881649100.5317
  - mission_id : variance = 2352.5835
  - real_x : variance = 95.9217
  - erreur_m : variance = 39.5477
  - velocity : variance = 14.0319
  - real_y : variance = 9.8684
  - real_z : variance = 8.4203
  - yaw : variance = 5.2019
  - az : variance = 1.2700
  - vx : variance = 0.5739
  - vz : variance = 0.3629
  - vy : variance = 0.3237
  - ay : variance = 0.1766
  - ax : variance = 0.0911
  - wind_y : variance = 0.0720
  - wind_x : variance = 0.0713
  - fog : variance = 0.0163
  - snow : variance = 0.0144


In [12]:
import pandas as pd

# Charger le fichier
df = pd.read_csv("donnees_vol_multiple_filtered_bon.csv")

# Réinitialiser l'index
df = df.reset_index(drop=True)

# Garder la valeur is_attack originale par mission_id
is_attack_values = df.groupby('mission_id')['is_attack'].first().reset_index()

# Ajouter un compteur par mission_id (position du waypoint)
df['ligne_id'] = df.groupby('mission_id').cumcount()

# Passer de long à wide : chaque ligne devient un vol à plat
df_flat = df.pivot(index='mission_id', columns='ligne_id')

# Aplatir les colonnes multi-niveaux
df_flat.columns = [f"{col}_{i}" for col, i in df_flat.columns]

# Réinitialiser l'index pour fusionner
df_flat = df_flat.reset_index()

# Fusionner avec les valeurs originales de is_attack
df_flat = pd.merge(df_flat, is_attack_values, on='mission_id', how='left')

# Supprimer toutes les colonnes is_attack_X qui ont été créées par le pivot
cols_to_drop = [col for col in df_flat.columns if col.startswith('is_attack_')]
df_flat = df_flat.drop(columns=cols_to_drop)

# Sauvegarder le fichier aplati
df_flat.to_csv("vols_aplatis_BON.csv", index=False)

print("✅ Vols aplatis avec succès ! (avec un seul is_attack par vol)")

✅ Vols aplatis avec succès ! (avec un seul is_attack par vol)
